## Постановка задачи
Загрузим данные изображений, построим на них размеченные области, сформируем массив серых пикселей из изображений.

Выгрузим массив пикселей в HDF5 для дальнейшей работы.

Данные:
* https://video.ittensive.com/machine-learning/clouds/train.csv.gz (54 Мб)
* https://video.ittensive.com/machine-learning/clouds/train_images_small.tar.gz (212 Мб)

Соревнование: https://www.kaggle.com/c/understanding_cloud_organization/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
from PIL import Image

### Загрузка данных

In [3]:
train = pd.read_csv("https://video.ittensive.com/machine-learning/clouds/train.csv.gz")
print (train.head())

          Image_Label                                      EncodedPixels
0    0011165.jpg_Fish  264918 937 266318 937 267718 937 269118 937 27...
1  0011165.jpg_Flower  1355565 1002 1356965 1002 1358365 1002 1359765...
2  0011165.jpg_Gravel                                                NaN
3   0011165.jpg_Sugar                                                NaN
4    002be4f.jpg_Fish  233813 878 235213 878 236613 878 238010 881 23...


### Очистка данных
Отделим категории и область каждой формы облаков

In [4]:
train["Image"] = train["Image_Label"].str.split("_").str[0]
train["Label"] = train["Image_Label"].str.split("_").str[1]
train.drop(labels=["Image_Label"], axis=1, inplace=True)
print (train.head(20))

                                        EncodedPixels        Image   Label
0   264918 937 266318 937 267718 937 269118 937 27...  0011165.jpg    Fish
1   1355565 1002 1356965 1002 1358365 1002 1359765...  0011165.jpg  Flower
2                                                 NaN  0011165.jpg  Gravel
3                                                 NaN  0011165.jpg   Sugar
4   233813 878 235213 878 236613 878 238010 881 23...  002be4f.jpg    Fish
5   1339279 519 1340679 519 1342079 519 1343479 51...  002be4f.jpg  Flower
6                                                 NaN  002be4f.jpg  Gravel
7   67495 350 68895 350 70295 350 71695 350 73095 ...  002be4f.jpg   Sugar
8   3510 690 4910 690 6310 690 7710 690 9110 690 1...  0031ae9.jpg    Fish
9   2047 703 3447 703 4847 703 6247 703 7647 703 9...  0031ae9.jpg  Flower
10                                                NaN  0031ae9.jpg  Gravel
11  658170 388 659570 388 660970 388 662370 388 66...  0031ae9.jpg   Sugar
12                       

In [5]:
data = pd.DataFrame({"Image": train["Image"].unique()})
for l in train["Label"].unique():
    data[l] = pd.Series(train[train["Label"] == l]["EncodedPixels"].values)
print (data.head())

         Image                                               Fish  \
0  0011165.jpg  264918 937 266318 937 267718 937 269118 937 27...   
1  002be4f.jpg  233813 878 235213 878 236613 878 238010 881 23...   
2  0031ae9.jpg  3510 690 4910 690 6310 690 7710 690 9110 690 1...   
3  0035239.jpg                                                NaN   
4  003994e.jpg  2367966 18 2367985 2 2367993 8 2368002 62 2369...   

                                              Flower  \
0  1355565 1002 1356965 1002 1358365 1002 1359765...   
1  1339279 519 1340679 519 1342079 519 1343479 51...   
2  2047 703 3447 703 4847 703 6247 703 7647 703 9...   
3  100812 462 102212 462 103612 462 105012 462 10...   
4                                                NaN   

                                              Gravel  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3  65400 380 66800 380 6

### Обработка изображений
Предварительно загрузим весь архив с изображениями и распакуем его в train_images_small

Пример: 0011165.jpg, размеры 525 * 350 = 183750 пикселей
![](train_images_small/0011165.jpg)

Каждое изображение приведем к серой палитре, результат загрузим в фрейм данных.

In [6]:
imgdata = np.array([np.zeros(183750, dtype="uint8")]*len(data))
for i, img in enumerate(data["Image"].unique()):
    imgdata[i] = np.array(Image.open("train_images_small/" +
                img).convert("L"), dtype="uint8").reshape(1, -1)[0]
imgdata = pd.DataFrame(imgdata)
print (imgdata.head())

   0       1       2       3       4       5       6       7       8       \
0       0       0       0       0       0       0       0       0       0   
1      71      42      31      38      48      68      70      46      24   
2      97      98     100     102     105     107     109     110     111   
3      87      86      89      93      93      88      86      86      81   
4      18      19      21      21      20      19      17      16      14   

   9       ...  183740  183741  183742  183743  183744  183745  183746  \
0       0  ...      82      65      52      70      64     102     113   
1      21  ...      98     113     110      59      27      38      57   
2     112  ...       0       0       0       0       0       0       0   
3      81  ...     181     187     170     149     145     165     164   
4      15  ...      66      75     112     156     140     125     137   

   183747  183748  183749  
0     102      96     116  
1      82     110     133  
2       

In [7]:
for column in data.columns:
    imgdata[column] = data[column]
del data
print (imgdata.head())

    0   1    2    3    4    5    6    7    8    9  ...  183745  183746  \
0   0   0    0    0    0    0    0    0    0    0  ...     102     113   
1  71  42   31   38   48   68   70   46   24   21  ...      38      57   
2  97  98  100  102  105  107  109  110  111  112  ...       0       0   
3  87  86   89   93   93   88   86   86   81   81  ...     165     164   
4  18  19   21   21   20   19   17   16   14   15  ...     125     137   

   183747  183748  183749        Image  \
0     102      96     116  0011165.jpg   
1      82     110     133  002be4f.jpg   
2       0       0       0  0031ae9.jpg   
3     108      70      72  0035239.jpg   
4      92      59      48  003994e.jpg   

                                                Fish  \
0  264918 937 266318 937 267718 937 269118 937 27...   
1  233813 878 235213 878 236613 878 238010 881 23...   
2  3510 690 4910 690 6310 690 7710 690 9110 690 1...   
3                                                NaN   
4  2367966 18 2367985 

### Сохраняем данные в HDF5
Потребуется до 3 Гб оперативной памяти

In [8]:
imgdata.to_hdf("clouds.data.h5", "data", format="fixed",
              compression="gzip", complevel=9, mode="w")

c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\pandas\io\pytables.py:274: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['Image', 'Fish', 'Flower', 'Gravel', 'Sugar']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
